In [2]:
from __future__ import print_function
from builtins import input
import csv
import spiceypy
import numpy as np
import pandas as pd
import datetime
from astropy.time import Time
import requests

def getsta(mag_time):
        #
        # Local parameters
        #
        METAKR = 'getgll.tm'
        #
        # Load the kernels that this program requires.  We
        # will need a leapseconds kernel to convert input
        # UTC time strings into ET.  We also will need the
        # necessary SPK files with coverage for the bodies
        # in which we are interested.
        #
        spiceypy.furnsh( METAKR )

        #
        #Prompt the user for the input time string.
        # 
        # start_time_str = '1996-6-11 19:32:00'
        row = []
        for t in mag_time:

                # start_time = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S")

                # utctim = start_time + datetime.timedelta(minutes=i*1000)

                time_string = t.strftime('%Y-%m-%d %H:%M:%S')

                #ephemeris time after iterating
                et = spiceypy.str2et( time_string )

                #
                # Compute the apparent state of GLL as seen from
                # Jupiter in the J2000 frame.  All of the ephemeris
                # readers return states in units of kilometers and
                # kilometers per second.
                #
                [state, ltime] = spiceypy.spkezr( '5', et,      'J2000',
                                                'LT+S',   '-77'       )
                row.append([state[0], state[1], state[2], state[3], state[4], state[5]]) 
        return row
                
     
 #
 # Fetch PDS Data for MAG
 #                
def get_data(url):
        parent_response = requests.get(url)

        if parent_response.status_code == 200:
                print('Successfully loaded JSON')
                parent_data = parent_response.json()
                parent_urlkey = parent_data['properties']["ops:Data_File_Info.ops:file_ref"][0]
                tab_urls_res = requests.get(parent_urlkey)
                tab_urls_cont = tab_urls_res.text.splitlines()
                #Define mag arrays
                mag_time = []
                mag_tot = []

                for i in range(len(tab_urls_cont)):
                        if 'irc' in tab_urls_cont[i].split(',')[1]: #check for body null rate measurements
                            time_data, iter_data = get_tabs(tab_urls_cont[i].split(',')[1]) # get rid of column in csv
                            mag_time.append(time_data)
                            mag_tot.append(iter_data)

                # for i in range(1): #range(len(data['data'])):
                #         data_url = data['properties']["ops:Data_File_Info.ops:file_ref"][0]#data['data'][i]['properties']["ops:Data_File_Info.ops:file_ref"][0]
                #         mag_data = requests.get(data_url)
                #         mag_content = mag_data.text
                #         lines = mag_content.splitlines()

                #         for line in lines:
                #                 raw_data = line.split()
                #                 mag_time.append(raw_data[0])
                #                 mag_tot.append(raw_data[4])

        return mag_time, mag_tot


def get_tabs(url):
        #load lid found from parent collection
        lid_response = requests.get('https://pds.nasa.gov/api/search/1/products/'+url)
        lid_data = lid_response.json()

        time_data = []
        iter_data = []
        for i in range(1): #range(len(lid_data['data'])):
                tab_url = lid_data['properties']['ops:Data_File_Info.ops:file_ref'][0]
                #load in tab data
                tab_response = requests.get(tab_url)
                tab_content = tab_response.text
                lines = tab_content.splitlines()

                for line in lines:
                        raw_data = line.split()
                        time_data.append(raw_data[0])
                        iter_data.append(raw_data[4])
        return time_data, iter_data

def remove_non_unique_values(data):
    counts = {}
    for item in data:
        counts[item] = counts.get(item, 0) + 1

    unique_values = [item for item, count in counts.items() if count == 1]
    return unique_values


if __name__ == '__main__':
       
       url = "https://pds.nasa.gov/api/search/1/products/urn:nasa:pds:galileo-mag-jup-calibrated:data-highres-magnetosphere::1.0"
       mag_time, mag_tot = get_data(url)

        #Remove duplicates and maintain order of mag_data
       df = pd.DataFrame({'time':mag_time, 'data': mag_tot})
       df['time'] = pd.to_datetime(df['time'])
       #now sort values
       df.sort_values('time', inplace = True)
       df.drop_duplicates(subset='time', keep ='first', inplace = True)

       df.reset_index(drop=True, inplace=True)
        #Obtain the position data that corresponds to the measurements
       mag_ephem_data = getsta(df['time']) #Takes time input
       print(len(mag_ephem_data))


       #Fill in the gaps for the ephemeris data
       ephem_time = []
       start_time = df['time'][0]
       for i in range(3000):
                # start_time = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S")
                ephem_time.append(start_time + datetime.timedelta(minutes=i*1000))   
       #Grab the states
       ephem_data = getsta(ephem_time)

       #Create csv for ephemeris data
       eph_file = 'data/eph_data2.csv'
       with open(eph_file, 'w',newline='') as file:
                writer1 = csv.writer(file)
                writer1.writerow(['Time', 'X', 'Y', 'Z']) 
                for i in range(len(ephem_data)):
                        writer1.writerow([ephem_time[i].replace(tzinfo=None).isoformat(), float(ephem_data[i][0]), 
                                          float(ephem_data[i][1]), float(ephem_data[i][2])])  
                         
       csv_file = 'data/tot_data2.csv'  
       with open(csv_file, 'w',newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Time','Magnitude (nT)', 'X', 'Y', 'Z'])
                for i in range(len(mag_ephem_data)):
                        writer.writerow([df['time'][i].replace(tzinfo=None).isoformat(), float(df['data'][i]),
                                          float(mag_ephem_data[i][0]), float(mag_ephem_data[i][1]),
                                            float(mag_ephem_data[i][2])])   
        
print('Done!')

Successfully loaded JSON


TypeError: <class 'list'> is not convertible to datetime, at position 0